# Clase Práctica #5 (Compilación)

En esta clase estaremos adaptando el evaluador de expresiones aritméticas para trabajar sobre un _AST (Abstact Syntax Tree)_. Recordemos que el AST posee una estructura más cómoda para evaluar las reglas semánticas que el árbol de derivación. Además, evaluar en el AST en lugar que desde las reglas de la gramática atributada, simplifica significativamente la implementación de las reglas semánticas.

## Jerarquía del AST

Definamos una jerarquía de clases para los nodos del _AST_ del lenguaje de expresiones aritméticas. Utilizaremos las clases `Node` y `BinaryNode` como definiciones abstractas para agrupar y compactar la implementación de sus descendientes. Los nodos del AST serán exclusivamente instancias de `ConstantNumberNode`, `PlusNode`, `MinusNode`, `StarNode` y `DivNode`.

In [1]:
class Node:
  def evaluate(self):
    raise NotImplementedError()

class ConstantNumberNode(Node):
  def __init__(self, lex):
    self.lex = lex
    self.value = float(lex)
  def evaluate(self):
    return self.value

class BinaryNode(Node):
  def __init__(self, left, right):
    self.left = left
    self.right = right
  def evaluate(self):
    lvalue = self.left.evaluate()
    rvalue = self.right.evaluate()
    return self.operate(lvalue, rvalue)
  @staticmethod
  def operate(lvalue, rvalue):
    raise NotImplementedError()

class PlusNode(BinaryNode):
  @staticmethod
  def operate(lvalue, rvalue):
    return lvalue + rvalue

class MinusNode(BinaryNode):
  @staticmethod
  def operate(lvalue, rvalue):
    return lvalue - rvalue

class StarNode(BinaryNode):
  @staticmethod
  def operate(lvalue, rvalue):
    return lvalue * rvalue

class DivNode(BinaryNode):
  @staticmethod
  def operate(lvalue, rvalue):
    return lvalue / rvalue


Veamos como luce una instancia concreta de un AST de expresiones. Nótese que la precedencia de los operadores debe seguir atrapada en el AST puesto que solo se desecharon los atributos sintácticos.

In [2]:
from cmp.ast import get_printer
printer = get_printer(AtomicNode=ConstantNumberNode, BinaryNode=BinaryNode)

print(printer(
  PlusNode(
    MinusNode(
      ConstantNumberNode('5'),
      ConstantNumberNode('6')
    ), ConstantNumberNode('9')
  )
))

\__<expr> PlusNode <expr>
	\__<expr> MinusNode <expr>
		\__ ConstantNumberNode: 5
		\__ ConstantNumberNode: 6
	\__ ConstantNumberNode: 9


## Construcción del AST

Pasemos a definir la gramática del lenguaje de expresiones aritméticas junto con las reglas semánticas para formar el _AST_. Las reglas quedarán muy similares a las de la clase anterior, pero esta vez en lugar de operar los valores, construiremos el nodo del AST que denota la operación.

In [3]:
from cmp.pycompiler import Grammar
from cmp.utils import pprint, inspect

G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y = G.NonTerminals('T F X Y')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) num')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
# ======================================================================= #
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]                    

# ======================================================================= #
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
# ======================================================================= #
X %= plus + T + X, lambda h,s: s[3], None, None, lambda h,s: PlusNode(h[0],s[2])
X %= minus + T + X, lambda h,s: s[3], None, None, lambda h,s: MinusNode(h[0],s[2])                            
X %= G.Epsilon, lambda h,s: h[0]                                           

# ======================================================================= #      
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
# ======================================================================= #
T %= F + Y, lambda h,s: s[2], None, lambda h,s: s[1]                       

# ======================================================================= #                   
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
# ======================================================================= #
Y %= star + F + Y, lambda h,s: s[3], None, None, lambda h,s: StarNode(h[0],s[2])                            
Y %= div + F + Y, lambda h,s: s[3], None, None, lambda h,s: DivNode(h[0],s[2])                            
Y %= G.Epsilon, lambda h,s: h[0]                                           

# ======================================================================= #       
#                                                                         #
# ======================= { F --> num | ( E ) } ========================= #
#                                                                         #
# ======================================================================= #
F %= num, lambda h,s: ConstantNumberNode(s[1]), None                                                    
F %= opar + E + cpar, lambda h,s: s[2], None, None, None                   

############################# END PRODUCTIONS #############################

Ensamblemos el pipeline de evaluación con los elementos que hemos ido implementando a lo largo de las pasadas clases.

Se realizará la siguiente cadena de transformaciones:
```
Entrada -> Tokens -> Parse Izquierdo -> AST -> Resultado
```    

In [4]:
from cmp.utils import Token
from cmp.languages import BasicHulk
from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
from cmp.tools.evaluation import evaluate_parse

hulk = BasicHulk(G)
firsts = hulk.firsts
follows = hulk.follows
tokenize_text = hulk.tokenizer

M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M)

def run_pipeline(text, value, parser, formatter):
  tokens = tokenize_text(text)
  pprint(tokens, '================Tokens================')
  left_parse = parser(tokens)
  pprint(left_parse, '==============Left-Parse==============')
  ast = evaluate_parse(left_parse, tokens)
  pprint(formatter(ast), '=================AST==================')
  result = ast.evaluate()
  pprint(f'{text} = {result}', '================Result================')
  assert result == value

Comprobemos que la asociatividad de los operadores no se perdió.

In [5]:
run_pipeline('1 - 1 - 1', -1, parser, printer)

================Tokens================
[
   num: 1
   -: -
   num: 1
   -: -
   num: 1
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> num
   Y -> e
   X -> e
]
=================AST==================
\__<expr> MinusNode <expr>
	\__<expr> MinusNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 1
	\__ ConstantNumberNode: 1
================Result================
1 - 1 - 1 = -1.0


In [6]:
run_pipeline('1 - ( 1 - 1 )', 1, parser, printer)

================Tokens================
[
   num: 1
   -: -
   (: (
   num: 1
   -: -
   num: 1
   ): )
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> ( E )
   E -> T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> num
   Y -> e
   X -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> MinusNode <expr>
	\__ ConstantNumberNode: 1
	\__<expr> MinusNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 1
================Result================
1 - ( 1 - 1 ) = 1.0


## Adicionando operador _potencia_

Añadamos el operador potencia al lenguaje. Para ello, realizaremos las modificaciones pertinentes a cada una de las fases del evaluador. No será necesario **copia y pegar** código de otras clases, ni modificar el código fuente del módulo `cmp` que se distribuye junto al _notebook_.

Usaremos el símbolo `^` para denotar al operador potencia. Este es un operador binario que computa $a^b$ siendo, `a` y `b` los operandos izquierdo y derecho respectivamente. Por ejemplo, `2 ^ 4` computa $2^4$. El operador potencia asocia hacia la derecha (contrario a los operadores: +, -, \* y /). Por tanto, `4 ^ 3 ^ 2` computa $4^{3^2} = 4^9$ en lugar de $(4^3)^2 = 4^6$.

In [7]:
class PowNode(BinaryNode):
  @staticmethod
  def operate(lvalue, rvalue):
    return lvalue ** rvalue

In [8]:
A, Z = G.NonTerminals('A Z')
pow = G.Terminal('^')

G.Productions.pop()
G.Productions.pop()

F %= A + Z, lambda h,s: s[2], None, lambda h,s: s[1]

Z %= pow + F, lambda h,s: PowNode(h[0],s[2]), None, None
Z %= G.Epsilon, lambda h,s: h[0]

A %= num, lambda h,s: ConstantNumberNode(s[1]), None                                                    
A %= opar + E + cpar, lambda h,s: s[2], None, None, None

In [9]:
hulk.fixed_tokens['^'] = Token('^', pow)

In [10]:
print(G)

Non-Terminals:
	E, T, F, X, Y, A, Z
Terminals:
	+, -, *, /, (, ), num, ^
Productions:
	[E -> T X, X -> + T X, X -> - T X, X -> e, T -> F Y, Y -> * F Y, Y -> / F Y, Y -> e, F -> A Z, Z -> ^ F, Z -> e, A -> num, A -> ( E )]


Como ayuda se proveen los conjunto _First_ y _Follow_ precomputados de lo que consideramos la gramática **natural** a obtener. De igual forma se tiene la tabla y parser LL(1). Siéntase libre de utilizar el código siguiente pero puede reemplazarlo por sus propias implementaciones en caso de que no pueda (o no entienda) cómo utilizarlo.

In [11]:
from cmp.languages import PowHulk
pow_hulk = PowHulk(G)

firsts = pow_hulk.firsts
follows = pow_hulk.follows

from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M)

**Comprobemos que el operador potencia asocia hacia la derecha.**

In [12]:
run_pipeline('4 ^ 3 ^ 2', 262144, parser, printer)

================Tokens================
[
   num: 4
   ^: ^
   num: 3
   ^: ^
   num: 2
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> ^ F
   F -> A Z
   A -> num
   Z -> ^ F
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PowNode <expr>
	\__ ConstantNumberNode: 4
	\__<expr> PowNode <expr>
		\__ ConstantNumberNode: 3
		\__ ConstantNumberNode: 2
================Result================
4 ^ 3 ^ 2 = 262144.0


**Comprobemos que tiene más precedencia que el resto de los operadores.**

In [13]:
run_pipeline('2 * 3 ^ 4 + 1 * 5', 167, parser, printer)

================Tokens================
[
   num: 2
   *: *
   num: 3
   ^: ^
   num: 4
   +: +
   num: 1
   *: *
   num: 5
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> * F Y
   F -> A Z
   A -> num
   Z -> ^ F
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> + T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> * F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PlusNode <expr>
	\__<expr> StarNode <expr>
		\__ ConstantNumberNode: 2
		\__<expr> PowNode <expr>
			\__ ConstantNumberNode: 3
			\__ ConstantNumberNode: 4
	\__<expr> StarNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 5
================Result================
2 * 3 ^ 4 + 1 * 5 = 167.0


**Comprobemos que puede subordinarse a otros operadores usando paréntesis.**

In [14]:
run_pipeline('3 ^ ( 1 + 1 ) ^ 2', 81, parser, printer)

================Tokens================
[
   num: 3
   ^: ^
   (: (
   num: 1
   +: +
   num: 1
   ): )
   ^: ^
   num: 2
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> ^ F
   F -> A Z
   A -> ( E )
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> + T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
   Z -> ^ F
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PowNode <expr>
	\__ ConstantNumberNode: 3
	\__<expr> PowNode <expr>
		\__<expr> PlusNode <expr>
			\__ ConstantNumberNode: 1
			\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 2
================Result================
3 ^ ( 1 + 1 ) ^ 2 = 81.0


## Adicionando _declaraciones de variables_

Añadamos variables al lenguaje de expresiones para acercarnos más a `HULK`. Para ello, agregue la expresión `let-in` al lenguaje. Dicha expresión sigue la siguiente sintaxis:

```
let
    <declaration-list>
in
    <expr>
```
donde `<expr>` denota cualquier expresión del lenguaje (incluyendo el propio `let-in`) y `<declaration-list>` representa una secuencia de declaraciones de la forma `<id> = <expr>` separadas por "`,`".

El valor de evaluación de la expresión `let-in` es el valor de evaluación de `<expr>`.  
Las variables declaradas en `<declaration-list>` serán visibles a partir de su declaración pero únicamente dentro de la expresión `let-in` que las contiene (incluye `<expr>`). Si `<expr>` contiene a su vez una expresión `let-in`, la declaración de una variable con el mismo nombre que una en el `let-in` padre **ocultará** la del padre.  
Por ejemplo, la expresión:
```
let
    x = 1,
    y = 2
in
    3 + (let x = 4, z = 5 in x + y + z) + x
```
equivalente a:
```
let x = 1, y = 2 in 3 + (let x = 4, z = 5 in x + y + z) + x
```
evalúa `15`.

In [15]:
class VariableNode(Node):    
  def __init__(self, lex):
    self.lex = lex
    
class DeclarationListNode(Node):
  def __init__(self, var, exp, next):
    self.var = var
    self.exp = exp
    self.next = next        

  def evaluate(self):
    try:
      dict = self.next.evaluate()
    except:
      dict = {}
    dict[self.var] = self.exp.evaluate()
    return dict    

class LetInNode(Node):
  def __init__(self, decl_list, exp_tree):
    self.decl_list = decl_list
    self.exp_tree = exp_tree
    self.var_dict = decl_list.evaluate()
    
  def evaluate(self):        
    new_exp_tree = self.sust_vars(self.exp_tree, self.var_dict)
    return new_exp_tree.evaluate()

    
  @staticmethod
  def sust_vars(exp_tree,var_dict):
    from copy import copy

    if isinstance(exp_tree, VariableNode):
      return ConstantNumberNode(var_dict[exp_tree.lex])
    elif isinstance(exp_tree, BinaryNode):
      left = LetInNode.sust_vars(exp_tree.left, var_dict)
      right = LetInNode.sust_vars(exp_tree.right, var_dict)
      return type(exp_tree)(left,right)
    elif isinstance(exp_tree, LetInNode):
      new_var_dict = copy(var_dict)
      for item in exp_tree.var_dict:
        new_var_dict[item] = exp_tree.var_dict[item]
      return LetInNode.sust_vars(exp_tree.exp_tree, new_var_dict)
    else:
      return exp_tree

In [16]:
# E --> LetIn
# A --> id
# LetIn --> let DeclarationList in Expression
# DeclarationList --> Declaration NextDeclarations
# Declaration --> id = Expression
# NextDeclaration --> , DeclarationList | e

let, id, equal, iN, comma = G.Terminals('let id = in ,')
LI, DL, D, ND = G.NonTerminals('LI DL D ND')

E %= LI, lambda h,s: s[1]

A %= id, lambda h,s: VariableNode(s[1]), None

LI %= let + DL + iN + E, lambda h,s: LetInNode(s[2],s[4]), None, None, None, None

DL %= D + ND, lambda h,s: DeclarationListNode(s[1][0],s[1][1],s[2]), None, None

D %= id + equal + E, lambda h,s: (s[1],s[3]), None, None, None

ND %= comma + DL, lambda h,s: s[2], None, None
ND %= G.Epsilon, lambda h,s: h[0]


In [17]:
def tokenize_text(text):
  tokens = []
  for item in text.split():

    try:
      float(item)
      token = Token(item, G['num'])
    except ValueError:
      try:
        token = fixed_tokens[item]
      except:
        token = Token(item, G['id'])
    
    tokens.append(token)
  
  eof = Token('$', G.EOF)
  tokens.append(eof)

  return tokens

fixed_tokens = { lex: Token(lex, G[lex]) for lex in '+ - * / ( ) ^ let = in ,'.split() }


from cmp.utils import ContainerSet

def compute_local_first(firsts, alpha):
  first_alpha = ContainerSet()

  try:
    alpha_is_epsilon = alpha.IsEpsilon
  except:
    alpha_is_epsilon = False    

  if alpha_is_epsilon:
    first_alpha.set_epsilon()   
  else:
    first_alpha.update(firsts[alpha._symbols[0]])
    i = 0
    xi = alpha._symbols[i]

    while firsts[xi].contains_epsilon:
      if i == len(alpha._symbols):
        first_alpha.set_epsilon()
        break
      i += 1
      xi = alpha._symbols[i]
      if not firsts[xi].contains_epsilon:
        first_alpha.update(firsts[xi])  
        break    

  return first_alpha

def compute_firsts(G):
  firsts = {}
  change = True  

  for terminal in G.terminals:
    firsts[terminal] = ContainerSet(terminal)
  for nonterminal in G.nonTerminals:
    firsts[nonterminal] = ContainerSet()

  while change:
    change = False       
    for production in G.Productions:
      X = production.Left
      alpha = production.Right            
      first_X = firsts[X]       

      try:
        first_alpha = firsts[alpha]
      except KeyError:
        first_alpha = firsts[alpha] = ContainerSet()    

      local_first = compute_local_first(firsts, alpha)            
      change |= first_alpha.hard_update(local_first)
      change |= first_X.hard_update(local_first)     

  return firsts

def compute_follows(G, firsts):
  follows = { }
  change = True    
  local_firsts = {}   
  
  for nonterminal in G.nonTerminals:
    follows[nonterminal] = ContainerSet()
  follows[G.startSymbol] = ContainerSet(G.EOF)
  
  while change:
    change = False      
    for production in G.Productions:
      X = production.Left
      alpha = production.Right
      follow_X = follows[X]            
      if alpha.IsEpsilon:
        continue            
      n = len(alpha._symbols)-1

      for i in range(n):
        Y = alpha._symbols[i]
        beta = alpha._symbols[i+1]

        if Y.IsNonTerminal:
          change |= follows[Y].update(firsts[beta])
          if firsts[beta].contains_epsilon:
            change |= follows[Y].update(follow_X)
        if i == n-1 and beta.IsNonTerminal:
          change |= follows[beta].update(follow_X)     
               
  return follows

In [18]:
firsts = compute_firsts(G)
follows = compute_follows(G, firsts)
M = build_parsing_table(G,firsts,follows)

parser = metodo_predictivo_no_recursivo(G, M)

def run_pipeline(text, value, parser, formatter):
  tokens = tokenize_text(text)
  pprint(tokens, '================Tokens================')
  left_parse = parser(tokens)
  pprint(left_parse, '==============Left-Parse==============')
  ast = evaluate_parse(left_parse, tokens)    
  result = ast.evaluate()
  pprint(f'{text} = {result}', '================Result================')
  assert result == value

run_pipeline('let x = 1 , y = 2 in 3 + ( let x = 4 , z = 5 in x + y + z ) + x', 15, parser, printer)

================Tokens================
[
   let: let
   id: x
   =: =
   num: 1
   ,: ,
   id: y
   =: =
   num: 2
   in: in
   num: 3
   +: +
   (: (
   let: let
   id: x
   =: =
   num: 4
   ,: ,
   id: z
   =: =
   num: 5
   in: in
   id: x
   +: +
   id: y
   +: +
   id: z
   ): )
   +: +
   id: x
   $: $
]
==============Left-Parse==============
[
   E -> LI
   LI -> let DL in E
   DL -> D ND
   D -> id = E
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
   ND -> , DL
   DL -> D ND
   D -> id = E
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
   ND -> e
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> + T X
   T -> F Y
   F -> A Z
   A -> ( E )
   E -> LI
   LI -> let DL in E
   DL -> D ND
   D -> id = E
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X -> e
   ND -> , DL
   DL -> D ND
   D -> id = E
   E -> T X
   T -> F Y
   F -> A Z
   A -> num
   Z -> e
   Y -> e
   X 